In [1]:
import numpy as np

In [101]:
def Entropy(p2, p1):
    ans = 0
    if p1 !=0:
        ans += - p1*np.log2(p1) 
    if p2 != 0:
        ans += - p2*np.log2(p2)
    return ans

In [109]:
Entropy(3/4, 1/4)

0.8112781244591328

In [103]:
0.94 - (8/14*Entropy(4/7, 3/7) + 7/14*Entropy(6/7, 1/7) )

0.08117610440354961

In [57]:
import pandas as pd

In [58]:
df = pd.read_csv('../dataset/titanic.csv')

In [64]:
np.unique([each for each in df['Survived']], return_counts=True)

(array([0, 1]), array([549, 342]))

In [62]:
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [65]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [71]:
not_useful = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
useful = df.drop(columns=not_useful)

In [72]:
useful.shape

(891, 7)

In [73]:
X = useful.values[:, 1:]
Y = useful.values[:, ]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [74]:
from sklearn.preprocessing import LabelEncoder

In [81]:
le = LabelEncoder()
useful['Sex'] = le.fit_transform(useful['Sex'])


In [83]:
useful.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [86]:
useful.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [93]:
useful = useful.fillna(useful.Age.mean()) # pass dict for multiple cols

In [94]:
useful.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(5)
memory usage: 48.8 KB


### Decision Tree

In [118]:
def entropy(col):
    counts = np.unique(col, return_counts=True)
    ent = 0.0
    N = float(col.shape[0])
    for ix in counts[1]:
        p = ix/N
        ent += -(p*np.log2(p))
    return ent

In [122]:
entropy(useful['Pclass'])

1.4393214704441286

In [156]:
def split_data(X_data, fkey, fval):
    # Binary split
    X_right = pd.DataFrame([], columns=X_data.columns)
    X_left = pd.DataFrame([], columns=X_data.columns)
    for ix in range(X_data.shape[0]):
        val = X_data[fkey].loc[ix]
        
        if val > fval:
            X_right = X_right.append(X_data.loc[ix])
        else:
            X_left = X_left.append(X_data.loc[ix])
            
    return X_right, X_left

In [157]:
R, L = split_data(useful, 'Sex',0.5)

In [158]:
R.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0.0,3.0,1.0,22.000000,1.0,0.0,7.2500
4,0.0,3.0,1.0,35.000000,0.0,0.0,8.0500
5,0.0,3.0,1.0,29.699118,0.0,0.0,8.4583
6,0.0,1.0,1.0,54.000000,0.0,0.0,51.8625
7,0.0,3.0,1.0,2.000000,3.0,1.0,21.0750


In [159]:
L.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
1,1.0,1.0,0.0,38.0,1.0,0.0,71.2833
2,1.0,3.0,0.0,26.0,0.0,0.0,7.9250
3,1.0,1.0,0.0,35.0,1.0,0.0,53.1000
8,1.0,3.0,0.0,27.0,0.0,2.0,11.1333
9,1.0,2.0,0.0,14.0,1.0,0.0,30.0708


In [163]:
def info_gain(X_data, fkey, fval):
    
    right, left = split_data(X_data, fkey, fval)
    
    l = float(left.shape[0]/X_data.shape[0])
    r = float(right.shape[0]/X_data.shape[0])
    
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -100000
    
    i_gain = entropy(X_data.Survived) - l*entropy(left.Survived)  - r*entropy(right.Survived)
    
    return i_gain
    

In [164]:
info_gain(useful, 'Pclass', 0.5)

-100000

In [190]:
X = useful[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

In [202]:
useful

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.000000,1,0,7.2500
1,1,1,0,38.000000,1,0,71.2833
2,1,3,0,26.000000,0,0,7.9250
3,1,1,0,35.000000,1,0,53.1000
4,0,3,1,35.000000,0,0,8.0500
5,0,3,1,29.699118,0,0,8.4583
6,0,1,1,54.000000,0,0,51.8625
7,0,3,1,2.000000,3,1,21.0750
8,1,3,0,27.000000,0,2,11.1333
9,1,2,0,14.000000,1,0,30.0708


In [174]:
for fx in X:
    print(fx)
    print(info_gain(useful, fx, useful[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606143
Age
0.001158644038169343
SibSp
0.009584541813400127
Parch
0.015380754493137888
Fare
0.04214069283899541


In [178]:
class DecisionTree:
    
    def __init__(self, depth = 0, max_depth = 5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        
    def train(self, X_train):
        
        features = ['Pclass', 'Sex', 'Age', 'SibSb', 'Parch', 'Fare']
        info_gain = []
        
        for ix in features:
            i_gain =info_gain(X_train, ix, X_train[ix].mean())
            info_gain.append(i_gain)
            
        selected =  features[np.argmax(info_gain)]
        self.fkey = selected
        self.fval = X_train[selected].mean()
            
        data_left, data_right = split_data(X_train, self.fkey, self.fval)
            
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
        
        # Base Case (Leaf Node)
        if (data_left.shape[0]==0 or data_right.shape[0] == 0 or self.depth == self.max_depth):
            if(X_train.Survived.mean() > 0.5):
                self.target = 1
            else:
                self.target = 0
                
        # Recursion Case
        # Create a left node 
        self.left = DecisionTree(depth=self.depth+1, max_depth=self.max_depth)
        #recursively build the left sub tree
        self.left.train(data_left)
        
        # Create a left node 
        self.right = DecisionTree(depth=self.depth+1, max_depth= self.max_depth)
        #recursively build the left sub tree
        self.right.train(data_right)
        
        
        
        
    def predict(self,  test):
        if test[self.fkey]>self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)
            
            
        
        

In [179]:
X.shape

(891, 6)

In [203]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(useful, test_size = 0.30)

In [204]:
type(x_train)

pandas.core.frame.DataFrame

In [206]:
dt = DecisionTree()

In [209]:
print(dt.fkey)

None


In [210]:
from sklearn.tree import DecisionTreeClassifier 

In [217]:
model = DecisionTreeClassifier()

In [219]:

X = useful[:, 1:]
Y = useful[:, 0]

In [220]:
from sklearn.model_selection import train_test_split

In [221]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [222]:
model.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [233]:
model.score(x_test, y_test)

0.8100558659217877

(891,)